# Air Quality in Istanbul between 2014.01 - 2020.07

 -*- coding: utf-8 -*-
"""
Created on Thursday Aug  2 21:34:41 2020
@author: erolerdogan
"""

**T	:**Average Temperature (°C)

**TM:**Maximum temperature (°C)

**Tm:**Minimum temperature (°C)

**SLP:**	Atmospheric pressure at sea level (hPa)

**H	:** Average relative humidity (%)

**PP:**Total rainfall and / or snowmelt (mm)

**VV:**Average visibility (Km)

**V	:**Average wind speed (Km/h)

**VM:**Maximum sustained wind speed (Km/h)

**VG:**Maximum speed of wind (Km/h)

**RA:**Indicate if there was rain or drizzle (In the monthly average, total days it rained)

**SN:**Snow indicator (In the monthly average, total days that snowed)

**TS:**Indicates whether there storm (In the monthly average, Total days with thunderstorm)

**FG:**Indicates whether there was fog (In the monthly average, Total days with fog)

### Data Sources

1. AQI (PM2.5, PM10 etc) => https://aqicn.org/data-platform/register/
2. The data between 2017-2020 from Turkey => https://sim.csb.gov.tr/
3. Details (T, TM, Tm etc.) => https://en.tutiempo.net/istanbul.html
4. http://www.havaizleme.gov.tr/


### Useful Links to understand project items better

1. PM2.5 vs PM10 => https://smartairfilters.com/en/blog/pm10-pm2-5-difference-particle-air-pollution/
2. Conversion from PM10 to PM2.5 => https://www.epd.gov.hk/epd/english/environmentinhk/air/guide_ref/guide_aqa_model_g5.html
3. Equation of calculating Air Quality Index => https://en.wikipedia.org/wiki/Air_quality_index

## Data Importing and Understanding

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

#### Air Quality Dataset from 1st source 

In [2]:
aqi_first = pd.read_csv("istanbul_aqi.csv", names=["Date", "pm2.5", "pm10", "o3", "no2", "so2", "co"], header=0)

aqi_first.head()

,Date,pm2.5,pm10,o3,no2,so2,co
0,2020/8/1,48,14,18,16,2,2
1,2020/8/2,42,11,17,11,1,1
2,2020/8/3,36,10,12,14,1,1
3,2020/8/4,28,9,,,,
4,2020/8/5,35,,,,,


#### Air Quality 2017-2020 dataset from Turkey ministry 2nd source

In [30]:
aqi_second = pd.read_excel("istanbul_besiktas_aqi_sim_2017-2020.xlsx", header=1,
                           names=["Date", "pm10", "so2", "co", "no2", "o3", "pm2.5"])
aqi_second.head()

,Date,pm10,so2,co,no2,o3,pm2.5
0,2017-01-02 00:00:56,NaN,13.10,436.68,100.37,20.86,NaN
1,2017-01-03 00:00:56,48.73,15.94,452.92,109.53,26.32,NaN
2,2017-01-04 00:00:56,58.86,12.93,585.75,120.80,16.18,NaN
3,2017-01-05 00:00:56,34.29,6.60,535.62,119.70,19.18,NaN
4,2017-01-06 00:00:56,30.91,6.57,457.47,91.04,24.16,NaN


#### Detailed Dataset (T, TM, Tm, PP etc.) from 3rd source

In [4]:
detailed_data = pd.read_csv("combined_data.csv")

detailed_data.head()

,Unnamed: 0,Day,T,TM,Tm,SLP,H,PP,VV,V,VM,VG,RA,SN,TS,FG,Date
0,0,1,7.7,9.0,6.3,NaN,84.0,0.00,7.1,11.5,18.3,NaN,,,,,2014-1
1,1,2,8.8,11.0,7.0,NaN,73.0,0.00,9.8,8.7,16.5,NaN,,,,,2014-1
2,2,3,9.0,11.0,7.5,NaN,83.0,0.51,8.4,10.2,14.8,NaN,NaN,,,,2014-1
3,3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,2014-1
4,4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,2014-1


#### 2017-11 dataset (due to absence of this month)

In [5]:
data_201701 = pd.read_excel("istanbul_mecidiyekoy_aqi_2017-11.xlsx", header=3,
                            names=["Date", "pm10", "so2", "co", "no2", "o3", "pm2.5"])
data_201701.head()

,Date,pm10,so2,co,no2,o3,pm2.5
0,2017-11-01 00:00:56,46.91,NaN,356.11,72.09,174.21,NaN
1,2017-11-02 00:00:56,66.23,NaN,840.17,89.83,272.77,NaN
2,2017-11-03 00:00:56,67.95,NaN,840.88,75.24,195.60,NaN
3,2017-11-04 00:00:56,38.48,NaN,529.11,71.66,167.90,NaN
4,2017-11-05 00:00:56,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleansing and Manipulation

#### Working on Detailed Data

In [6]:
for i in range(len(detailed_data)):
    detailed_data.loc[i, "Date"] = str(detailed_data.Date[i]) + "-" + str(detailed_data.Day[i])

detailed_data_df = detailed_data.iloc[:, 2:-6].copy().join(detailed_data.iloc[:, -1])
detailed_data_df.Date = pd.to_datetime(detailed_data_df.Date)
detailed_data_df.head()

,T,TM,Tm,SLP,H,PP,VV,V,VM,Date
0,7.7,9.0,6.3,NaN,84.0,0.00,7.1,11.5,18.3,2014-01-01
1,8.8,11.0,7.0,NaN,73.0,0.00,9.8,8.7,16.5,2014-01-02
2,9.0,11.0,7.5,NaN,83.0,0.51,8.4,10.2,14.8,2014-01-03
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-04
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-05


In [7]:
detailed_data_df.shape

(2404, 10)

In [9]:
detailed_data_df.isna().sum()

T       1181
TM      1181
Tm      1181
SLP     2401
H       1183
PP      1217
VV      1311
V       1181
VM      1181
Date       0
dtype: int64

In [10]:
detailed_data_df.notna().sum()

T       1223
TM      1223
Tm      1223
SLP        3
H       1221
PP      1187
VV      1093
V       1223
VM      1223
Date    2404
dtype: int64

#### Working on Air Quality Data from 1st Source

In [11]:
# Changing empty or wrong cell's values as NaN values
aqi_first.replace(["", " "], np.nan, inplace=True)

# Converting Date to actual datetime
aqi_first.Date = pd.to_datetime(aqi_first.Date)

aqi_first.sort_values(by="Date", inplace=True)
aqi_first.drop(0, inplace=True)
aqi_first.reset_index(drop=True, inplace=True)

# Removing 2020 datas because of we don't have it within 'combined_data.csv'
aqi_first = aqi_first.iloc[:-3,:]

# Due to combine data
mask = aqi_first.Date < "2017-01-02"
aqi_first = aqi_first[mask]

# Converting Date to actual datetime
aqi_first.Date = pd.to_datetime(aqi_first.Date).dt.strftime("%Y-%m-%d")
aqi_first.head()

,Date,pm2.5,pm10,o3,no2,so2,co
0,2013-12-31,NaN,NaN,NaN,28,1,5
1,2014-01-01,NaN,18,NaN,27,2,7
2,2014-01-02,NaN,27,NaN,31,3,9
3,2014-01-03,NaN,31,NaN,30,2,8
4,2014-01-04,NaN,27,NaN,30,2,10


In [12]:
aqi_first.shape

(1002, 7)

In [13]:
aqi_first.isnull().sum()

Date       0
pm2.5    868
pm10      39
o3       852
no2       65
so2      154
co       156
dtype: int64

In [14]:
# Dropping to PM2.5 index due to many NA values
#aqi.drop("pm2.5", axis=1, inplace=True)

aqi_first.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1002 entries, 0 to 1001
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    1002 non-null   object
 1   pm2.5   134 non-null    object
 2   pm10    963 non-null    object
 3   o3      150 non-null    object
 4   no2     937 non-null    object
 5   so2     848 non-null    object
 6   co      846 non-null    object
dtypes: object(7)
memory usage: 62.6+ KB


#### Working on Air Quality Dataset from 2nd source

In [15]:
aqi_second.Date = pd.to_datetime(aqi_second.Date).dt.strftime("%Y-%m-%d")

# Concantenating aqi data from second source and data of 2017-11 
aqi_second = pd.concat([aqi_second, data_201701])

aqi_second.head()

,Date,pm10,so2,co,no2,o3,pm2.5
0,2017-01-02,NaN,13.10,436.68,100.37,20.86,NaN
1,2017-01-03,48.73,15.94,452.92,109.53,26.32,NaN
2,2017-01-04,58.86,12.93,585.75,120.80,16.18,NaN
3,2017-01-05,34.29,6.60,535.62,119.70,19.18,NaN
4,2017-01-06,30.91,6.57,457.47,91.04,24.16,NaN


In [31]:
aqi_second.tail()

,Date,pm10,so2,co,no2,o3,pm2.5
1318,2020-08-12 00:00:56,NaN,NaN,NaN,NaN,NaN,NaN
1319,2020-08-13 00:00:56,19.01,3.87,585.56,42.83,NaN,NaN
1320,2020-08-14 00:00:56,14.00,2.73,295.08,35.61,NaN,NaN
1321,2020-08-15 00:00:56,14.75,4.20,236.73,33.66,NaN,NaN
1322,2020-08-16 00:00:56,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
aqi_second.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1355 entries, 0 to 31
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1355 non-null   object 
 1   pm10    1222 non-null   float64
 2   so2     1195 non-null   float64
 3   co      1281 non-null   float64
 4   no2     1125 non-null   float64
 5   o3      1188 non-null   float64
 6   pm2.5   0 non-null      float64
dtypes: float64(6), object(1)
memory usage: 84.7+ KB


#### Combining whole Air Quality Indexes Datasets

In [17]:
# Combined Istanbul Ministry data and website data (first source and second source datasets)
aqi = pd.concat([aqi_first, aqi_second])

# Converting the numeric values to float64 from object
aqi[["pm2.5", "pm10", "o3", "no2", "so2", "co"]] = aqi[["pm2.5", "pm10", "o3",
                                                                    "no2", "so2", "co"]].astype(dtype="float64", 
                                                                                                           errors="ignore")
aqi.Date = pd.to_datetime(aqi.Date)
aqi.head()

,Date,pm2.5,pm10,o3,no2,so2,co
0,2013-12-31,NaN,NaN,NaN,28.0,1.0,5.0
1,2014-01-01,NaN,18.0,NaN,27.0,2.0,7.0
2,2014-01-02,NaN,27.0,NaN,31.0,3.0,9.0
3,2014-01-03,NaN,31.0,NaN,30.0,2.0,8.0
4,2014-01-04,NaN,27.0,NaN,30.0,2.0,10.0


In [18]:
aqi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2357 entries, 0 to 31
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2357 non-null   datetime64[ns]
 1   pm2.5   134 non-null    float64       
 2   pm10    2185 non-null   float64       
 3   o3      1338 non-null   float64       
 4   no2     2062 non-null   float64       
 5   so2     2043 non-null   float64       
 6   co      2127 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 147.3 KB


In [19]:
aqi.isna().sum()

Date        0
pm2.5    2223
pm10      172
o3       1019
no2       295
so2       314
co        230
dtype: int64

#### Combining Air Quality Index Dataset and 

In [20]:
# Merging all data. Air Quality Dataset - Climate Information Dataset
combined_df = pd.merge(aqi, detailed_data_df, on="Date", how="right")

#combined_df.set_index([combined_df.Date], inplace=True)
combined_df.sort_values(by="Date", inplace=True)
combined_df.Date = pd.to_datetime(combined_df.Date).dt.strftime('%Y-%m')
combined_df.drop(["SLP", "o3", "pm2.5"], axis=1, inplace=True)
#combined_df.dropna(how="all", inplace=True)

# creating Month column to use later
for i in range(len(combined_df)):
    combined_df.loc[i, "Month"] = combined_df.Date[i].split("-")[1]
    combined_df.loc[i, "Year"] = combined_df.Date[i].split("-")[0]

In [21]:
combined_df.head()

,Date,pm10,no2,so2,co,T,TM,Tm,H,PP,VV,V,VM,Month,Year
0,2014-01,18.0,27.0,2.0,7.0,7.7,9.0,6.3,84.0,0.00,7.1,11.5,18.3,01,2014
1,2014-01,27.0,31.0,3.0,9.0,8.8,11.0,7.0,73.0,0.00,9.8,8.7,16.5,01,2014
2,2014-01,31.0,30.0,2.0,8.0,9.0,11.0,7.5,83.0,0.51,8.4,10.2,14.8,01,2014
3,2014-01,27.0,30.0,2.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01,2014
4,2014-01,46.0,34.0,3.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01,2014


In [22]:
for date in set(list(combined_df.Date)):   
    for column in combined_df.columns[1:-2]:
        mask = combined_df.Date == date
        mean = combined_df[mask][column].mean()
        
        #if str(mean) == "nan":
        #    print(" {}'s column {} is completely NaN".format(date, column))
            
        combined_df.loc[mask, column] = combined_df.loc[mask, column].fillna(mean)

**Important:** Based on the checking NaN values, I observed, it corresponds to almost 10 months in 2018 and 2019 years. Although it will cost me almost 500 days, I will remove it. Because I can't fill 10 months by looking only 2 months in a year.

In [23]:
combined_df.tail()

,Date,pm10,no2,so2,co,T,TM,Tm,H,PP,VV,V,VM,Month,Year
2303,2020-07,23.47,52.290000,2.76,245.10,25.6000,30.00000,21.00000,71.00,0.0,9.5000,11.7000,20.6000,07,2020
2304,2020-07,20.19,37.900000,2.07,297.20,25.1875,28.88125,21.75625,64.75,0.0,9.9875,19.0375,29.6375,07,2020
2305,2020-07,20.70,66.490000,3.43,343.52,25.1875,28.88125,21.75625,64.75,0.0,9.9875,19.0375,29.6375,07,2020
2306,2020-07,23.54,35.920000,1.96,405.20,25.1875,28.88125,21.75625,64.75,0.0,9.9875,19.0375,29.6375,07,2020
2307,2020-07,15.35,42.979655,4.32,347.25,27.3000,32.00000,23.50000,65.00,0.0,10.0000,13.5000,20.6000,07,2020


In [24]:
combined_df.isna().sum()

Date      0
pm10     30
no2      30
so2      61
co       30
T         0
TM        0
Tm        0
H         0
PP        0
VV        0
V         0
VM        0
Month     0
Year      0
dtype: int64

In [25]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2404 entries, 0 to 2307
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2404 non-null   object 
 1   pm10    2374 non-null   float64
 2   no2     2374 non-null   float64
 3   so2     2343 non-null   float64
 4   co      2374 non-null   float64
 5   T       2404 non-null   float64
 6   TM      2404 non-null   float64
 7   Tm      2404 non-null   float64
 8   H       2404 non-null   float64
 9   PP      2404 non-null   float64
 10  VV      2404 non-null   float64
 11  V       2404 non-null   float64
 12  VM      2404 non-null   float64
 13  Month   2404 non-null   object 
 14  Year    2404 non-null   object 
dtypes: float64(12), object(3)
memory usage: 380.5+ KB


In [26]:
grouped_combined_df = combined_df.groupby(["Date"], sort=False).first()
grouped_combined_df.head(15)

,pm10,no2,so2,co,T,TM,Tm,H,PP,VV,V,VM,Month,Year
Date,,,,,,,,,,,,,,
2014-01,18.000000,27.000000,2.000000,7.000000,7.7,9.0,6.3,84.0,0.00,7.100,11.5,18.3,01,2014
2014-02,15.000000,34.500000,1.000000,7.000000,5.7,7.2,3.0,71.0,0.00,10.000,22.6,31.7,02,2014
2014-03,41.275862,37.033333,2.366667,4.107143,10.6,14.0,8.2,72.0,0.25,9.800,2.6,14.8,03,2014
2014-04,52.000000,41.000000,3.000000,5.000000,11.4,15.0,5.0,73.0,0.00,8.900,6.5,20.6,04,2014
2014-05,25.000000,47.000000,2.000000,1.000000,15.0,18.0,12.5,79.0,3.30,10.000,4.1,18.3,05,2014
2014-06,30.000000,37.000000,1.000000,2.000000,18.3,23.0,15.0,80.0,2.29,9.000,11.1,35.2,06,2014
2014-07,37.000000,37.000000,1.000000,2.000000,24.0,28.0,18.0,61.0,0.00,9.700,12.6,22.2,07,2014
2014-08,26.875000,35.000000,1.200000,3.800000,27.8,31.0,24.5,61.0,0.00,10.000,20.2,31.7,08,2014
2014-09,28.000000,39.000000,2.000000,4.000000,24.6,27.0,22.5,66.0,0.00,10.000,11.9,22.2,09,2014


In [27]:
month_mask = grouped_combined_df.Month == "11"
year_mask = grouped_combined_df.Year == "2017"
grouped_combined_df[month_mask]

,pm10,no2,so2,co,T,TM,Tm,H,PP,VV,V,VM,Month,Year
Date,,,,,,,,,,,,,,
2014-11,42.100000,34.666667,2.200000,2.761905,11.6,13.0,10.7,69.0,8.64,9.0,26.3,40.7,11,2014
2015-11,19.000000,30.000000,1.681818,6.000000,13.7,16.3,12.9,55.0,0.00,10.0,26.9,37.0,11,2015
2016-11,37.000000,55.000000,3.000000,5.000000,9.7,13.0,5.8,53.0,0.00,10.0,18.3,35.2,11,2016
2017-11,NaN,NaN,NaN,NaN,9.5,13.6,5.6,50.0,0.00,11.4,13.9,22.2,11,2017
2018-11,23.811379,83.910000,4.220000,664.730000,17.9,24.0,12.7,75.0,0.00,7.7,17.0,31.7,11,2018
2019-11,14.410000,46.840000,2.570000,616.120000,13.6,15.0,11.0,76.0,5.08,9.0,25.0,33.5,11,2019


In [28]:
# PM10 looks best to use of calculation Air Quality Index due less missed values.
# The equation of calculating AQI based on chosen concentration. 
# (AQI_high - AQI_low)/(PM10_high - PM10_low) * (PM10 - PM10_low) + AQI_low 

combined_df.isna().sum()

Date      0
pm10     30
no2      30
so2      61
co       30
T         0
TM        0
Tm        0
H         0
PP        0
VV        0
V         0
VM        0
Month     0
Year      0
dtype: int64

### The Necessary Values of Calculation Air Quality Index,

![AQI Values](AQI_values.png)

## EDA (Explanatory Data Analysis)

In [33]:
grouped_combined_df

,pm10,no2,so2,co,T,TM,Tm,H,PP,VV,V,VM,Month,Year
Date,,,,,,,,,,,,,,
2014-01,18.000000,27.000000,2.000000,7.000000,7.7,9.0,6.3,84.0,0.00,7.1,11.5,18.3,01,2014
2014-02,15.000000,34.500000,1.000000,7.000000,5.7,7.2,3.0,71.0,0.00,10.0,22.6,31.7,02,2014
2014-03,41.275862,37.033333,2.366667,4.107143,10.6,14.0,8.2,72.0,0.25,9.8,2.6,14.8,03,2014
2014-04,52.000000,41.000000,3.000000,5.000000,11.4,15.0,5.0,73.0,0.00,8.9,6.5,20.6,04,2014
2014-05,25.000000,47.000000,2.000000,1.000000,15.0,18.0,12.5,79.0,3.30,10.0,4.1,18.3,05,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03,21.320000,74.710000,2.470000,387.410000,6.8,11.0,1.4,79.0,3.05,9.7,6.3,11.1,03,2020
2020-04,25.451000,59.446667,1.420000,214.550000,7.9,9.0,6.0,80.0,0.00,10.0,22.8,31.7,04,2020
2020-05,18.580000,74.420000,6.700000,186.600000,16.7,21.0,14.0,78.0,0.00,10.0,9.6,16.5,05,2020
